In [1]:
import openai
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
# Add a constant to the model (intercept)
from sklearn.metrics import r2_score
import time
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import tiktoken

from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
'''If executed in Google Colab, uncomment the following lines'''
#from google.colab import drive
#drive.mount('/content/drive')

#import os
#os.chdir('/content/drive/MyDrive/LLM_CreditorRRPrediction')

In [ ]:
POST = True

if POST:
    aggregated = pd.read_csv('../transcripts/post_aggregated_credit_df.csv', delimiter='|')
else:
    aggregated = pd.read_csv('../transcripts/aggregated_credit_df.csv', delimiter='|')

#PART = 'presentation'
#PART = 'QnA'
PART = 'analysts'

In [ ]:
client = openai.OpenAI(api_key=OPENAI_API_KEY)

output = []
start = 0

# import system_message.txt as a string
with open(f'prompts/system_{PART}_section.txt', 'r') as file:
    system_message = file.read()


print(system_message)

In [ ]:
if PART == 'QnA' or PART == 'analysts':
    part = 'QnA'
else:
    part = PART


for idx, row in aggregated.iterrows():
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": row[part]}
        ]
    )
    # Instantly add the output as a new column entry for the corresponding row
    response = completion.choices[0].message.content

    print(f"Completed for row {idx}")
    
    # compute token count for the response
    # Initialize the OpenAI tokenizer
    encoding = tiktoken.encoding_for_model("gpt-4o-mini")

    # Function to calculate the number of tokens in a transcript using OpenAI's tokenizer
    def count_tokens(transcript):
        tokens = encoding.encode(transcript)
        return len(tokens)
    
    print(f"Token count: {count_tokens(response)}")

    aggregated.at[idx, f'{PART}_summary'] = response

In [ ]:
aggregated.head()

In [ ]:
# save as csv
if POST:
    aggregated.to_csv(f'../transcripts/post_{PART}_summary.csv', index=False, sep='|')
else:
    aggregated.to_csv(f'../transcripts/{PART}_summary.csv', index=False, sep='|')